In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
### Indexing 
import bs4
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

doc_path = "dev-data/Be_Good.pdf"
loader = PyPDFLoader(doc_path)

doc = loader.load()

# Split 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)

# Make splits
splits = text_splitter.split_documents(doc)

# Index
chromadb = Chroma.from_documents(
    documents=splits,
    embedding=GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
)
retriever = chromadb.as_retriever(search_kwargs={"k": 1})

In [3]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on
single input query.\nGenerate multiple search queries related to: {question}\n Output 
(4 queries):"""

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq

generate_queries = (
    prompt_rag_fusion
    | ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct")
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [5]:
question = "Who is the writter of the book?"
generate_queries.invoke({"question": question})

['Here are 4 multiple search queries related to "Who is the writer of the book?":',
 '',
 '1. **Author of the book**: Who wrote this book?',
 '2. **Book author search**: What is the name of the author of the book?',
 '3. **Writer of the book**: Who is the creator of this book?',
 '4. **Book writer query**: Can you identify the author of this book?',
 '',
 'Let me know if you need more!']

In [6]:
from langchain.load import dumps, loads 

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents
        and an optioan parameter k used in the RRF formula"""
    fused_scores = {}

    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

C:\Users\Tommy RAMAROKOTO\AppData\Local\Temp\ipykernel_7184\2401826664.py:16: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  (loads(doc), score)


8

In [9]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash-preview-05-20")


prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'The writer of the book is Paul Graham.'